In [ ]:
def register_update_tabs_callback(app, name):
    @app.callback(
        Output('file_tabs' + name, 'tabs'),
        Output('file_tabs' + name, 'children'),
        Input('uploaded-file' + name, 'data'),
        Input('file_tabs' + name, 'active_tab'),
    )
    def update_tabs(data, active_tab):
        if not data or not data[0]:
            return [], []

        encoded_files, file_texts, file_names = data
        num_files = len(encoded_files)

        tabs = [
            {'id': f'tab{i + 1}{name}', 'title': file_names[i]}
            for i in range(num_files)
        ]

        if not active_tab:
            active_tab = f'tab1{name}'

        try:
            tab_num = int(''.join(filter(str.isdigit, active_tab)))
            tab_id = tab_num - 1
        except (ValueError, TypeError):
            tab_id = 0

        tab_id = max(0, min(tab_id, len(encoded_files) - 1))

        # Simplified iframe implementation
        children = [
            dbc.Container([
                html.Iframe(
                    src=encoded_files[tab_id],
                    style={
                        'width': '100%',
                        'height': '800px',
                        'border': 'none'
                    }
                )
            ], fluid=True)
        ]

        return tabs, children

In [ ]:
def register_update_tabs_callback(app, name):
    @app.callback(
        Output('file_tabs' + name, 'tabs'),
        Output('file_tabs' + name, 'children'),
        Input('uploaded-file' + name, 'data'),
        Input('file_tabs' + name, 'active_tab'),
    )
    def update_tabs(data, active_tab):
        if not data or not data[0]:
            return [], []

        encoded_files, file_texts, file_names = data
        num_files = len(encoded_files)

        tabs = [
            {'id': f'tab{i + 1}{name}', 'title': file_names[i]}
            for i in range(num_files)
        ]

        if not active_tab:
            active_tab = f'tab1{name}'

        try:
            tab_num = int(''.join(filter(str.isdigit, active_tab)))
            tab_id = tab_num - 1
        except (ValueError, TypeError):
            tab_id = 0

        tab_id = max(0, min(tab_id, len(encoded_files) - 1))

        # Simplified iframe implementation
        children = [
            dbc.Container([
                html.Iframe(
                    src=encoded_files[tab_id],
                    style={
                        'width': '100%',
                        'height': '800px',
                        'border': 'none'
                    }
                )
            ], fluid=True)
        ]

        return tabs, children